In [1]:
import pandas as pd
import os
import xlrd
import re
path = '../data/combined_excel.xlsx'
df = pd.read_excel(path,index_col=0, engine='openpyxl')
df['50'] = df['50'].astype(str)

# 第一種case

In [2]:
case1 = {}
for k,v in zip(df['50'].index,df['50'].values):
    if "_x000D" in v:
        case1[k] = v.split('_x000D')[0][:]
res1 = pd.DataFrame()
res1['string_X_train'] = df.loc[case1.keys(),'50'].values
res1['Y_label'] = case1.values()
df = res1

def preprocess(x):
    x = str(x)
    x = re.sub('[\u4e00-\u9fa5]', '', x) # 去除中文
    x = re.sub(r'[^\w\s]','',x) # 去除標點符號
    x = x.replace('\n', '').replace('\r', '').replace('\t', '') # 換行符號去除
    return str.strip(x) # 移除左右空白

df['string_X_train'] = df['string_X_train'].apply(preprocess)
df['Y_label'] = df['Y_label'].apply(preprocess)
df.head()

,string_X_train,Y_label
0,HONGKONG PROANDHAR GROUP COMPANY_x000D_LIM ITED,HONGKONG PROANDHAR GROUP COMPANY
1,TRAFIGURA PTE LTD_x000D_10 COLLYER QUAY_x000D_...,TRAFIGURA PTE LTD
2,KINGFA SCI AND TECH CO LTD_x000D_ADDRESS NO33 ...,KINGFA SCI AND TECH CO LTD
3,LANDMARK INTL CORP_x000D_SUITE 310GRIFFITH COR...,LANDMARK INTL CORP
4,DVA PLASTICS GmbH_x000D_VALENTINSKAMP 70_x000D...,DVA PLASTICS GmbH


# 起始結束標註

In [3]:
def str2index(context,string):
    ys = context.find(string)
    ye = ys + len(string)
    return ys,ye

ys_lst = []
ye_lst = []
for i in range(len(df)):
    context= df['string_X_train'].values[i]
    string = df['Y_label'].values[i]
    ys,ye = str2index(context,string)
    ys_lst.append(ys)
    ye_lst.append(ye)

df['string_Y_1'] = ys_lst
df['string_Y_2'] = ye_lst
df.head()

,string_X_train,Y_label,string_Y_1,string_Y_2
0,HONGKONG PROANDHAR GROUP COMPANY_x000D_LIM ITED,HONGKONG PROANDHAR GROUP COMPANY,0,32
1,TRAFIGURA PTE LTD_x000D_10 COLLYER QUAY_x000D_...,TRAFIGURA PTE LTD,0,17
2,KINGFA SCI AND TECH CO LTD_x000D_ADDRESS NO33 ...,KINGFA SCI AND TECH CO LTD,0,26
3,LANDMARK INTL CORP_x000D_SUITE 310GRIFFITH COR...,LANDMARK INTL CORP,0,18
4,DVA PLASTICS GmbH_x000D_VALENTINSKAMP 70_x000D...,DVA PLASTICS GmbH,0,17


In [4]:
print(df.shape)
df.to_csv('../data/preprocess_for_SQUAD_開狀人.csv')

(12378, 4)


In [5]:
df['predict'] = [str.strip(i.split('_x000D')[0]) for i in df['string_X_train'].values]

In [6]:
for i in range(len(df)):
    if df['Y_label'].values[i] == df['predict'].values[i]:
        df.loc[i,'correct'] = 'yes'
    else:
        df.loc[i,'correct'] = 'no'
df.head()

,string_X_train,Y_label,string_Y_1,string_Y_2,predict,correct
0,HONGKONG PROANDHAR GROUP COMPANY_x000D_LIM ITED,HONGKONG PROANDHAR GROUP COMPANY,0,32,HONGKONG PROANDHAR GROUP COMPANY,yes
1,TRAFIGURA PTE LTD_x000D_10 COLLYER QUAY_x000D_...,TRAFIGURA PTE LTD,0,17,TRAFIGURA PTE LTD,yes
2,KINGFA SCI AND TECH CO LTD_x000D_ADDRESS NO33 ...,KINGFA SCI AND TECH CO LTD,0,26,KINGFA SCI AND TECH CO LTD,yes
3,LANDMARK INTL CORP_x000D_SUITE 310GRIFFITH COR...,LANDMARK INTL CORP,0,18,LANDMARK INTL CORP,yes
4,DVA PLASTICS GmbH_x000D_VALENTINSKAMP 70_x000D...,DVA PLASTICS GmbH,0,17,DVA PLASTICS GmbH,yes


In [7]:
df['correct'].value_counts()

yes    12378
Name: correct, dtype: int64